<a href="https://colab.research.google.com/github/mohamedtal/Master2021-AutoML/blob/main/24_04_2021_HPO_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Optimization (HPO)

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn import datasets
import scipy.stats as stats

## Load MNIST dataset

In [8]:
d = datasets.load_digits()
X = d.data
y = d.target

In [5]:
#ANN
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
def ANN(optimizer = 'sgd',neurons=32,batch_size=32,epochs=20,activation='relu',patience=3,loss='categorical_crossentropy'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(10,activation='softmax'))  
    model.compile(optimizer = optimizer, loss=loss)
    early_stopping = EarlyStopping(monitor="loss", patience = patience)# early stop patience
    history = model.fit(X, pd.get_dummies(y).values,
              batch_size=batch_size,
              epochs=epochs,
              callbacks = [early_stopping],
              verbose=0) 
    return model

## HPO Algorithm 1: Grid Search


In [18]:
import time
#SVM
from sklearn.model_selection import GridSearchCV
rf_params = {
    'C': [1,10, 100],
    "kernel":['linear','poly','rbf','sigmoid']
}
clf = SVC(gamma='scale')
grid = GridSearchCV(clf, rf_params, scoring='accuracy')
start = time.time()
grid.fit(X, y)
end = time.time()
print(grid.best_params_)
print("Accuracy:"+ str(grid.best_score_))
print("time:",end-start)



{'C': 10, 'kernel': 'rbf'}
Accuracy:0.9738502011761063
time: 5.006103277206421


In [19]:
#KNN
from sklearn.model_selection import GridSearchCV
rf_params = {
    'n_neighbors': [2, 3, 5,10,15,20],
}
clf = KNeighborsClassifier()
grid = GridSearchCV(clf, rf_params,  scoring='accuracy')
start = time.time()
grid.fit(X, y)
end = time.time()
print(grid.best_params_)
print("Accuracy:"+ str(grid.best_score_))
print("time:",end-start)

{'n_neighbors': 2}
Accuracy:0.9671711544413494
time: 2.2014286518096924


In [20]:
#ANN
from sklearn.model_selection import GridSearchCV
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [16,32],
    'neurons':[16,32],
    'epochs':[20,50],
    'patience':[2,5]
}
clf = KerasClassifier(build_fn=ANN, verbose=0)
grid = GridSearchCV(clf, rf_params,scoring='accuracy')
start = time.time()
grid.fit(X, y)
end = time.time()
print(grid.best_params_)
print("MSE:"+ str(grid.best_score_))
print("time:",end-start)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


{'activation': 'relu', 'batch_size': 16, 'epochs': 50, 'neurons': 32, 'optimizer': 'adam', 'patience': 5}
MSE:1.0
time: 4316.611445426941


## HPO Algorithm 2: Random Search


In [21]:
#SVM
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'C': stats.uniform(0,50),
    "kernel":['linear','poly','rbf','sigmoid']
}
n_iter_search=20
clf = SVC(gamma='scale')
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,scoring='accuracy')
start = time.time()
Random.fit(X, y)
end = time.time()
print(Random.best_params_)
print("Accuracy:"+ str(Random.best_score_))
print("time:",end-start)

{'C': 26.790207061258187, 'kernel': 'rbf'}
Accuracy:0.9738502011761063
time: 8.280427932739258


In [22]:
#KNN
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'n_neighbors': range(1,20),
}
n_iter_search=10
clf = KNeighborsClassifier()
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,scoring='accuracy')
start = time.time()
Random.fit(X, y)
end = time.time()
print(Random.best_params_)
print("Accuracy:"+ str(Random.best_score_))
print("time:",end-start)

{'n_neighbors': 1}
Accuracy:0.9643933766635715
time: 3.682990789413452


In [23]:
#ANN
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [16,32,64],
    'neurons':sp_randint(10,100),
    'epochs':[20,50],
    'patience':sp_randint(3,20)
}
n_iter_search=10
clf = KerasClassifier(build_fn=ANN, verbose=0)
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,scoring='accuracy')
start = time.time()
Random.fit(X, y)
end = time.time()
print(Random.best_params_)
print("Accuracy:"+ str(Random.best_score_))
print("time:",end-start)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


{'activation': 'tanh', 'batch_size': 16, 'epochs': 50, 'neurons': 85, 'optimizer': 'adam', 'patience': 10}
Accuracy:1.0
time: 485.5570909976959


## HPO Algorithm 4: BO-GP

### Using skopt.BayesSearchCV

In [2]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 6.8MB/s 


In [9]:
#SVM
import time
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'C': Real(0.01,50),
    "kernel":['linear','poly','rbf','sigmoid']
}
clf = SVC(gamma='scale')
Bayes = BayesSearchCV(clf, rf_params,n_iter=20, n_jobs=-1,scoring='accuracy')
start = time.time()
Bayes.fit(X, y)
end = time.time()
print(Bayes.best_params_)
bclf = Bayes.best_estimator_
print("Accuracy:"+ str(Bayes.best_score_))
print("time:",end-start)

OrderedDict([('C', 7.061357850299573), ('kernel', 'rbf')])
Accuracy:0.9744017807456873
time: 25.11559557914734


In [10]:
#KNN
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'n_neighbors': Integer(1,20),
}
clf = KNeighborsClassifier()
Bayes = BayesSearchCV(clf, rf_params,n_iter=10, n_jobs=-1,scoring='accuracy')
start = time.time()
Bayes.fit(X, y)
end = time.time()
print(Bayes.best_params_)
bclf = Bayes.best_estimator_
print("Accuracy:"+ str(Bayes.best_score_))
print("time:",end-start)

OrderedDict([('n_neighbors', 3)])
Accuracy:0.9660545353366722
time: 3.6921935081481934


In [11]:
#ANN
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'optimizer': ['adam','rmsprop','sgd'],
    'activation': ['relu','tanh'],
    'batch_size': [16,32,64],
    'neurons':Integer(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':Integer(3,20)
}
clf = KerasClassifier(build_fn=ANN, verbose=0)
Bayes = BayesSearchCV(clf, rf_params,n_iter=10, scoring='accuracy')
start = time.time()
Bayes.fit(X, y)
end = time.time()
print(Bayes.best_params_)
print("Accuracy:"+ str(Bayes.best_score_))
print("time:",end-start)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses

OrderedDict([('activation', 'relu'), ('batch_size', 16), ('epochs', 33), ('neurons', 66), ('optimizer', 'adam'), ('patience', 5)])
Accuracy:1.0
time: 421.5909068584442


## HPO Algorithm 5: BO-TPE

In [12]:
#SVM
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        'C': abs(float(params['C'])), 
        "kernel":str(params['kernel'])
    }
    clf = SVC(gamma='scale', **params)
    score = cross_val_score(clf, X, y, scoring='accuracy').mean()

    return {'loss':-score, 'status': STATUS_OK }

space = {
    'C': hp.normal('C', 0, 50),
    "kernel":hp.choice('kernel',['linear','poly','rbf','sigmoid'])
}

start = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20)
end = time.time()
print("SVM: Hyperopt estimated optimum {}".format(best))
print("time:",end-start)

100%|██████████| 20/20 [00:07<00:00,  2.69it/s, best loss: -0.9738502011761063]
SVM: Hyperopt estimated optimum {'C': -15.722002742139265, 'kernel': 2}
time: 7.447922945022583


In [13]:
#KNN
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        'n_neighbors': abs(int(params['n_neighbors']))
    }
    clf = KNeighborsClassifier( **params)
    score = cross_val_score(clf, X, y, scoring='accuracy').mean()

    return {'loss':-score, 'status': STATUS_OK }

space = {
    'n_neighbors': hp.quniform('n_neighbors', 1, 20, 1),
}
start = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
end = time.time()
print("KNN: Hyperopt estimated optimum {}".format(best))
print("time:",end-start)

100%|██████████| 10/10 [00:03<00:00,  2.52it/s, best loss: -0.9638409161250386]
KNN: Hyperopt estimated optimum {'n_neighbors': 4.0}
time: 3.9800121784210205


In [14]:
#ANN
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        "optimizer":str(params['optimizer']),
        "activation":str(params['activation']),
        'batch_size': abs(int(params['batch_size'])),
        'neurons': abs(int(params['neurons'])),
        'epochs': abs(int(params['epochs'])),
        'patience': abs(int(params['patience']))
    }
    clf = KerasClassifier(build_fn=ANN,**params, verbose=0)
    score = -np.mean(cross_val_score(clf, X, y, 
                                    scoring="accuracy"))

    return {'loss':score, 'status': STATUS_OK }

space = {
    "optimizer":hp.choice('optimizer',['adam','rmsprop','sgd']),
    "activation":hp.choice('activation',['relu','tanh']),
    'batch_size': hp.quniform('batch_size', 16, 64, 16),
    'neurons': hp.quniform('neurons', 10, 100, 10),
    'epochs': hp.quniform('epochs', 20, 50, 10),
    'patience': hp.quniform('patience', 3, 20, 3),
}

start = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
end = time.time()
print("ANN: Hyperopt estimated optimum {}".format(best))
print("time:",end-start)

  0%|          | 0/10 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '



100%|██████████| 10/10 [05:43<00:00, 34.36s/it, best loss: -1.0]
ANN: Hyperopt estimated optimum {'activation': 1, 'batch_size': 16.0, 'epochs': 30.0, 'neurons': 90.0, 'optimizer': 1, 'patience': 6.0}
time: 343.57102394104004
